In [1]:


import os
import json
import math
import time
import pickle
import random
import urllib
import catboost
import simdkalman

import pandas as pd
import requests as req

from copy import copy
from glob import glob
from tqdm import tqdm
from collections import Counter
from scipy.stats import entropy
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,mean_squared_error
from fuzzywuzzy import fuzz, process
from sklearn.metrics import make_scorer
from datetime import datetime, timedelta
from bayes_opt import BayesianOptimization
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict

from multiprocessing import Process

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


DROPBOX_PATH = 'C:\\Users\Blake\G Street Dropbox\Blake Atkinson\shared_soccer_data\data'

def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

teams = load_dict(os.path.join(DROPBOX_PATH, 'IDs/teams'))
competitions = load_dict(os.path.join(DROPBOX_PATH, 'IDs/competitions'))
def load_schedules():
    
    normal = pd.read_csv(os.path.join(DROPBOX_PATH, 'schedules/processed_schedule.csv'))
    stf = pd.read_csv(os.path.join(DROPBOX_PATH, 'schedules/stf_schedule.csv'))

    normal['home_team_name'] = normal['home_team_id'].apply(lambda x: teams.get(x)['name'])
    normal['away_team_name'] = normal['away_team_id'].apply(lambda x: teams.get(x)['name'])

    stf['team_name'] = stf['team_id'].apply(lambda x: teams.get(x)['name'])
    stf['opp_team_name'] = stf['opp_team_id'].apply(lambda x: teams.get(x)['name'])
    
    normal['datetime_UTC'] = pd.to_datetime(normal['datetime_UTC'].copy())
    stf['datetime_UTC'] = pd.to_datetime(stf['datetime_UTC'].copy())

    normal['match_date_UTC'] = normal['datetime_UTC'].copy().dt.date
    stf['match_date_UTC'] = stf['datetime_UTC'].copy().dt.date
    
    normal['last_updated'] = pd.to_datetime(normal['last_updated'].copy())
    stf['last_updated'] = pd.to_datetime(stf['last_updated'].copy())
    
    normal = normal.loc[~normal['match_status'].isin(['deleted','collecting','cancelled','postponed'])].reset_index(drop=True)
    stf = stf.loc[~stf['match_status'].isin(['deleted','collecting','cancelled','postponed'])].reset_index(drop=True)
    
    return normal, stf 

schedule, stf = load_schedules()





C:\Users\Blake\AppData\Local\Temp\ipykernel_28636\1142054372.py:50: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  normal = pd.read_csv(os.path.join(DROPBOX_PATH, 'schedules/processed_schedule.csv'))
C:\Users\Blake\AppData\Local\Temp\ipykernel_28636\1142054372.py:51: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  stf = pd.read_csv(os.path.join(DROPBOX_PATH, 'schedules/stf_schedule.csv'))


In [7]:
from pinnacle.apiclient import APIClient
api = APIClient("PWXTESTO27", "mmnn1122")
sport_ids = api.reference_data.get_sports()
tennis_events = api.market_data.get_fixtures(33)

In [8]:
tennis_events

{'TIMESTAMP': '2023-03-02 00:06:42.255718',
 'code': 'INVALID_CREDENTIALS',
 'message': 'Authorization failed, invalid credentials',
 'Latency': 0.153528}

In [10]:
user = "PWXTEST027"
password = "mmnn1122"

import base64
import requests
from enum import Enum


headers = {}
headers.update({'Accept': 'application/json'})
headers.update({'Content-Type': 'application/json'})
headers.update({'Authorization': 'Basic {}'.format(
    base64.b64encode((bytes("{}:{}".format(user, password), 'utf-8'))).decode())
})

operation = 'api.ps3838.com/v3/sports'
req = requests.get(
    url=operation
    headers=get_headers(HttpMethod.GET)
)



SyntaxError: invalid syntax (3138429853.py, line 19)

In [3]:

user = ''
pwd = ''

import base64
import requests
from enum import Enum

"""
Script to connect to ps3838 API

URL to check API User Guide:
https://www.tender88.com/static/index.php/es-es/help/api-user-guide-es-es

In order to access PS3838 API you must have a funded account.
"""

# API ENDPOINT
API_ENDPOINT = 'http://api.ps3838.com'


# Available Request Methods
class HttpMethod(Enum):
    GET = 'GET'
    POST = 'POST'


# Constants to fill by each user
PS3838_USERNAME = "PWXTEST027"
PS3838_PASSWORD = "mmnn1122"


def get_headers(request_method: HttpMethod) -> dict:
    headers = {}
    headers.update({'Accept': 'application/json'})
    if request_method is HttpMethod.POST:
        headers.update({'Content-Type': 'application/json'})

    headers.update({'Authorization': 'Basic {}'.format(
        base64.b64encode((bytes("{}:{}".format(PS3838_USERNAME, PS3838_PASSWORD), 'utf-8'))).decode())
    })

    return headers


def get_operation_endpoint(operation: str) -> str:
    return '{}{}'.format(API_ENDPOINT, operation)


def get_sports():
    operation = '/v3/sports'
    req = requests.get(
        get_operation_endpoint(operation),
        headers=get_headers(HttpMethod.GET)
    )
    print(req)
    return req.json()


# Test retrieve sports endpoint
print(get_sports())



<Response [403]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)